# The Delaunay Triangulation and the Flip Algorithm

## CS 480 Computational Geometry
### Dr. John C. Bowers

For this lab you are going to code the flip algorithm for computing a Delaunay triangulation. Recall that this algorithm starts with a given triangulation $T$. The algorithm then repeats the following steps until there are no illegal edges: find an illegal edge $e$ and flip it. That's it! 

As we saw in class, testing whether an edge is legal can be done using an `inCircle` predicate. If `ABC` and `BCD` are two triangles sharing an edge `BC` and `ABCD` is a convex quadrilateral, then `BC` is illegal if and only if `D` is incide the circle defined by `A`, `B`, and `C`. Otherwise, if `ABCD` is non-convex or `D` is outside the circle, then `BC` is legal. Similarly, any edge of the convex hull is legal.

# Predicates

The main new predicate here is the `inCircle` test which implements the algorithm we discussed in class which lifts the points to the upper paraboloid $z = x^2 + y^2$ and tests which side of the plane through lifted points $A$, $B$, and $C$ the point $D$ lies on. As we saw, above the plane is outside the circle and below the plane is inside the circle. 

In [117]:
from koebe.geometries.euclidean2 import PointE2, SegmentE2

def areaOfParallelogram(A: PointE2, B: PointE2, C: PointE2) -> float:
    return (B.x - A.x) * (C.y - A.y) - (C.x - A.x) * (B.y - A.y)

def areaOfTriangle(A: PointE2, B: PointE2, C: PointE2) -> float:
    return 0.5 * areaOfParallelogram(A, B, C)

def leftHandTurn(A: PointE2, B: PointE2, C: PointE2) -> bool:
    return areaOfTriangle(A, B, C) > 0

def inCircle(A: PointE2, B: PointE2, C: PointE2, D: PointE2) -> bool:
    """
    Returns true if D is in the circle through points A, B, and C. 
    """
    adx = A.x - D.x
    ady = A.y - D.y
    bdx = B.x - D.x
    bdy = B.y - D.y
    cdx = C.x - D.x
    cdy = C.y - D.y

    abdet = adx * bdy - bdx * ady
    bcdet = bdx * cdy - cdx * bdy
    cadet = cdx * ady - adx * cdy
    alift = adx * adx + ady * ady
    blift = bdx * bdx + bdy * bdy
    clift = cdx * cdx + cdy * cdy
    
    return (alift * bcdet + blift * cadet + clift * abdet) > 0

def orient(p1, p2, p3):
    return (p1, p2, p3) if leftHandTurn(p1, p2, p3) else (p1, p3, p2)

# Datastructures

We are continuing to use the DCEL structure that we saw in the last lab. You will implement two functions in the `HalfEdge` class. 

In [118]:
class DCEL:
    """
    The basic DCEL container. 
    
    Attributes:
        verts: List[Vertex] - The list of vertices. 
        halfEdges: List[HalfEdge] - The list of half-edges. 
        faces: List[Face] - The list of faces. 
        outerFace: Face - The outerFace (None if there isn't one.)
    """
    def __init__(self):
        self.verts = []
        self.halfEdges = []
        self.faces = []
        self.outerFace = None
    
class Vertex:
    """
    The basic Vertex object. 
    
    Attributes:
        dcel: DCEL - The parent DCEL. 
        outgoingHalfEdge: HalfEdge - Any one half-edge with this vertex as its origin.
        
    """
    def __init__(self, dcel, outgoingHalfEdge=None, pos=None):
        self.dcel = dcel
        self.dcel.verts.append(self)
        self.outgoingHalfEdge = outgoingHalfEdge
        self.pos = pos

class Face:
    def __init__(self, dcel, incidentHalfEdge=None):
        self.dcel = dcel
        self.dcel.faces.append(self)
        self.incidentHalfEdge = incidentHalfEdge
    
    def incidentHalfEdges(self):
        result = []
        curr = self.incidentHalfEdge
        while True:
            result.append(curr)
            curr = curr.next
            if curr == self.incidentHalfEdge:
                break
        return result
    
    def incidentVertices(self):
        return [he.origin for he in self.incidentHalfEdges]

class HalfEdge:
    def __init__(self, 
                 dcel, 
                 origin=None, 
                 face=None, 
                 prev=None, 
                 next=None, 
                 twin=None):
        
        self.dcel   = dcel
        self.dcel.halfEdges.append(self)
        
        self.origin = origin
        self.face   = face
        self.prev   = prev
        self.next   = next
        self.twin   = twin
    
    def makeNext(self, he):
        self.next = he
        he.prev = self
    
    def makeTwin(self, he):
        self.twin = he
        he.twin = self
    
    @property # The @property annotation let's you "run" this as
              # he.destination instead of he.destination()
    def destination(self):
        return self.twin.origin
    
    def isLegal(self) -> bool:
        return False # TODO
    
    def flip(self):
        # First check if this is a valid flippable edge (i.e. an illegal edge)
        # Then do the flip by fixing self and self.twin's pointers (and the surrounding
        # half-edges of the flip quadrilateral). HINT: Remember that .face and .origin
        # pointers need to be updated for self and self.twin and .face pointers need
        # to be updated for all the half-edges incident the outer quadrilateral. 
        pass # TODO

def splitFace(e1: HalfEdge, e2: HalfEdge) -> None:
    if e1.face != e2.face:
        return
    
    newFace = Face(e1.dcel)

    s1 = HalfEdge(dcel=e1.dcel, origin=e2.origin, face=e1.face)
    s2 = HalfEdge(dcel=e1.dcel, origin=e1.origin, face=newFace)
    s1.makeTwin(s2)

    e2.prev.makeNext(s1)
    e1.prev.makeNext(s2)
    s1.makeNext(e1)
    s2.makeNext(e2)

    newFace.incidentHalfEdge = e2
    e1.face.incidentHalfEdge = e1

    for e in e1.face.incidentHalfEdges(): e.face = e1.face
    for e in newFace.incidentHalfEdges(): e.face = newFace

# Algorithms

I have provided a working implementation of the incremental triangulation algorithm. You will implement `delaunify`, which takes in a triangulation and turns it into a Delaunay triangulation by exploring the flip graph. 

In [119]:
from typing import List

def triangleDCEL(p1: PointE2, p2: PointE2, p3: PointE2) -> DCEL:
    """
    Convenience method for creating a DCEL representing a triangle given its three corners. 
    """
    result = DCEL()
    result.outerFace = Face(result)
    
    tri = Face(result)
    
    A, B, C = [Vertex(dcel=result, pos=p) for p in orient(p1, p2, p3)]
    
    AB = HalfEdge(dcel=result, origin=A, face=tri)
    BC = HalfEdge(dcel=result, origin=B, face=tri)
    CA = HalfEdge(dcel=result, origin=C, face=tri)
    
    AC = HalfEdge(dcel=result, origin=A, face=result.outerFace)
    CB = HalfEdge(dcel=result, origin=C, face=result.outerFace)
    BA = HalfEdge(dcel=result, origin=B, face=result.outerFace)
    
    AB.makeTwin(BA)
    BC.makeTwin(CB)
    CA.makeTwin(AC)
    
    AB.makeNext(BC)
    BC.makeNext(CA)
    CA.makeNext(AB)
    
    AC.makeNext(CB)
    CB.makeNext(BA)
    BA.makeNext(AC)
    
    tri.incidentHalfEdge = AB
    result.outerFace.incidentHalfEdge = BA
    
    return result

def findLowerTangent(dcel, p):
    for he in dcel.outerFace.incidentHalfEdges():
        if (leftHandTurn(he.prev.origin.pos, he.origin.pos, p) 
            and leftHandTurn(he.next.origin.pos, he.origin.pos, p)):
            return he
    return None

def findUpperTangent(dcel, p):
    for he in dcel.outerFace.incidentHalfEdges():
        if (leftHandTurn(p, he.origin.pos, he.next.origin.pos) 
            and leftHandTurn(p, he.origin.pos, he.prev.origin.pos)):
            return he
    return None

def incrTriangulation(S: List[PointE2]) -> DCEL:
    thePoints = sorted(S, key=lambda p:(p.x, p.y))
    result = triangleDCEL(*thePoints[0:3])
    for p in thePoints[3:]:
        lowerTangent = findLowerTangent(result, p)
        upperTangent = findUpperTangent(result, p)
        
        upper_orig_prev = upperTangent.prev
        
        v = Vertex(dcel=result, pos=p)
        f = Face(dcel=result, incidentHalfEdge=upperTangent)

        upper_inner = HalfEdge(dcel=result, origin=v, face=f)
        lower_inner = HalfEdge(dcel=result, origin=lowerTangent.origin, face=f)
        
        upper_outer = HalfEdge(dcel=result, origin=upperTangent.origin, face=result.outerFace)
        lower_outer = HalfEdge(dcel=result, origin=v, face=result.outerFace)
        
        upper_inner.makeTwin(upper_outer)
        lower_inner.makeTwin(lower_outer)
        
        upper_inner.makeNext(upperTangent)
        lowerTangent.prev.makeNext(lower_inner)
        lower_inner.makeNext(upper_inner)
        
        upper_orig_prev.makeNext(upper_outer)
        upper_outer.makeNext(lower_outer)
        lower_outer.makeNext(lowerTangent)
        
        result.outerFace.incidentHalfEdge = upper_outer
        f.incidentHalfEdge = upperTangent
        v.outgoingHalfEdge = upper_inner
        
        for e in f.incidentHalfEdges(): e.face = f
        
        curr = upper_inner.next.next
        vHe = upper_inner
        while curr != lower_inner:
            curr = curr.next
            splitFace(curr.prev, vHe)
            vHe = curr.prev.prev
            
    return result
        
        
def delaunify(triangulation: DCEL):
    # Use your flip() code to convert the given triangulation into a DCEL
    # using the flip algorithm from class. 
    pass # TODO

# Graphics Library

Two new canvases based on the editable point set canvas we used in the prior labs. 

In [120]:
from ipycanvas import Canvas, hold_canvas

def IncrTriangulationFlipCanvas(size, pts, draw_func = None):
    
    points = list(pts)
    canvas = Canvas(size=size)
    selectedPointIdx = -1
    
    incr = incrTriangulation(points)
    for he in incr.halfEdges: he.selected = False
    
    def _canvas_draw():
        nonlocal canvas, points, selectedPointIdx, draw_func, incr
        
        with hold_canvas(canvas):
            canvas.clear()

            if draw_func != None:
                draw_func(canvas, points)
                
            for he in incr.halfEdges:
                he.marked = False

            for he in incr.halfEdges:
                if not he.marked:
                    if he.selected:
                        canvas.stroke_style = "blue"
                    else:
                        canvas.stroke_style = "black" if he.isLegal() else "red"
                    u = he.origin
                    v = he.destination
                    canvas.begin_path()
                    canvas.move_to(int(u.pos.x), int(u.pos.y))
                    canvas.line_to(int(v.pos.x), int(v.pos.y))
                    canvas.stroke()
                    he.marked = True
                    he.twin.marked = True

            canvas.fill_style = 'blue'
            canvas.fill_rects([round(p.x) - 4 for p in points], 
                              [round(p.y) - 4 for p in points], 
                              8)
            if selectedPointIdx != -1:
                canvas.fill_style = 'red'
                canvas.fill_rect(round(points[selectedPointIdx].x) - 4, 
                                 round(points[selectedPointIdx].y) - 4, 
                                 8)
            
    def handle_mouse_down(x, y):
        nonlocal selectedPointIdx, incr
        # See if any point is close to x, y
        cursorPoint = PointE2(x, y)
        sqDists = [p.distSqTo(cursorPoint) for p in points]
        minIdx = sqDists.index(min(sqDists))
        
        if sqDists[minIdx] < 24:
            selectedPointIdx = minIdx
        
        if selectedPointIdx == -1:
            # Then we will try selecting an edge to flip
            for he in incr.halfEdges: he.selected = False
            sqDists = [SegmentE2(he.origin.pos, he.destination.pos).distSqTo(cursorPoint)
                       for he in incr.halfEdges]
            minIdx = sqDists.index(min(sqDists))
            if sqDists[minIdx] < 24:
                incr.halfEdges[minIdx].selected = True
                incr.halfEdges[minIdx].twin.selected = True
                incr.halfEdges[minIdx].flip()
        
        _canvas_draw()

    def handle_mouse_up(x, y):
        nonlocal selectedPointIdx
        selectedPointIdx = -1 # No point is selected anymore.
        _canvas_draw()

    def handle_mouse_move(x, y):
        nonlocal selectedPointIdx, incr
        if selectedPointIdx >= 0:
            points[selectedPointIdx] = PointE2(x, y)
            incr = incrTriangulation(points)
            for he in incr.halfEdges: he.selected = False
        _canvas_draw()
    
    canvas.on_mouse_down(handle_mouse_down)
    canvas.on_mouse_up(handle_mouse_up)
    canvas.on_mouse_move(handle_mouse_move)
    
    _canvas_draw()
    
    return canvas


def DelaunayTriangulationFlipCanvas(size, pts, draw_func = None):
    
    points = list(pts)
    canvas = Canvas(size=size)
    selectedPointIdx = -1
    
    incr = incrTriangulation(points)
    delaunify(incr)
    
    def _canvas_draw():
        nonlocal canvas, points, selectedPointIdx, draw_func, incr
        
        with hold_canvas(canvas):
            canvas.clear()

            if draw_func != None:
                draw_func(canvas, points)
                
            for he in incr.halfEdges:
                he.marked = False

            for he in incr.halfEdges:
                if not he.marked:
                    u = he.origin
                    v = he.destination
                    canvas.begin_path()
                    canvas.move_to(int(u.pos.x), int(u.pos.y))
                    canvas.line_to(int(v.pos.x), int(v.pos.y))
                    canvas.stroke()
                    he.marked = True
                    he.twin.marked = True

            canvas.fill_style = 'blue'
            canvas.fill_rects([round(p.x) - 4 for p in points], 
                              [round(p.y) - 4 for p in points], 
                              8)
            if selectedPointIdx != -1:
                canvas.fill_style = 'red'
                canvas.fill_rect(round(points[selectedPointIdx].x) - 4, 
                                 round(points[selectedPointIdx].y) - 4, 
                                 8)
            
    def handle_mouse_down(x, y):
        nonlocal selectedPointIdx, incr
        # See if any point is close to x, y
        cursorPoint = PointE2(x, y)
        sqDists = [p.distSqTo(cursorPoint) for p in points]
        minIdx = sqDists.index(min(sqDists))
        
        if sqDists[minIdx] < 24:
            selectedPointIdx = minIdx
        
        _canvas_draw()

    def handle_mouse_up(x, y):
        nonlocal selectedPointIdx
        selectedPointIdx = -1 # No point is selected anymore.
        _canvas_draw()

    def handle_mouse_move(x, y):
        nonlocal selectedPointIdx, incr
        if selectedPointIdx >= 0:
            points[selectedPointIdx] = PointE2(x, y)
            incr = incrTriangulation(points)
            delaunify(incr)
        _canvas_draw()
    
    canvas.on_mouse_down(handle_mouse_down)
    canvas.on_mouse_up(handle_mouse_up)
    canvas.on_mouse_move(handle_mouse_move)
    
    _canvas_draw()
    
    return canvas

# Your Tasks

Your task is to implement the following three functions:

* `HalfEdge.isLegal`: Returns True if and only if the half-edge represents a legal edge in the triangulation (as per the Delaunay definition of "legal").
* `HalfEdge.flip`: Flips the edge represented by the half-edge if it is flippable.
* `delaunify`: Takes as input a DCEL that is a triangulation. Turns it into a Delaunay triangulation using the flip algorithm. 

__Turning it in:__ Turn everything in to Canvas by the deadline.


# Applications

## Flip editor

The following application let's you edit a point set _and_ flip edges by clicking on them. Legal Delaunay edges are drawn in black, illegal edges are in red, and if you flip an edge the flipped edge will be drawn in green. You can use this to test your `isLegal` and `flip` code as you work. 

In [121]:
IncrTriangulationFlipCanvas(
    size=(800, 800),
    pts=[
        PointE2(p.x * 400 + 200, p.y * 400 + 200)
        for p in [PointE2(0.1, 0.29),
         PointE2(0.3, 0.23), 
         PointE2(0.55, 0.43), 
         PointE2(0.31, 0.61), 
         PointE2(0.31, 0.45), 
         PointE2(0.68, 0.52), 
         PointE2(0.69, 0.17)]
    ]
)

Canvas(layout=Layout(height='800px', width='800px'), size=(800, 800))

## Delaunay Triangulation


The following application is useful once you finish `delaunify`. It will allow you to drag the points of a point set around and show you the Delaunay triangulation of the points. 

In [122]:
DelaunayTriangulationFlipCanvas(
    size=(800, 800),
    pts=[
        PointE2(p.x * 400 + 200, p.y * 400 + 200)
        for p in [PointE2(0.1, 0.29),
         PointE2(0.3, 0.23), 
         PointE2(0.55, 0.43), 
         PointE2(0.31, 0.61), 
         PointE2(0.31, 0.45), 
         PointE2(0.68, 0.52), 
         PointE2(0.69, 0.17)]
    ]
)

Canvas(layout=Layout(height='800px', width='800px'), size=(800, 800))